# loading data and creating methods

In [1]:
import pyreason as pr
from loramon import TrainerBase, TrainerBDIF, do_battle, TrainerLLMFeed, TrainerLLMFeedSC, TrainerLLM
from tools.pick_six import agent_create_team
from itertools import permutations, combinations
from sim.sim import dict_to_team_set
import google.generativeai as genai
from collections import Counter
from scipy.stats import binomtest
import os

os.environ["API_KEY"] = open("api_key.txt").read()
genai.configure(api_key=os.environ["API_KEY"])

c:\Users\zahdehv\Documents\GITHUB Media\pokemon-python\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def do_matches(p1: TrainerBase,p2: TrainerBase, debug = False, n = 6):
    teams = []
    for _ in range(n):
        teams.append(dict_to_team_set(agent_create_team()))
    matches = list(permutations(teams, 2))

    victor = []

    for i, mtch in enumerate(matches):
        print("mtch:", i) #, mtch)
        p1.team = mtch[0]
        p2.team = mtch[1]
        try:
            _, winner = do_battle(p1, p2, debug=debug)

            if winner:
                victor.append(winner.name)
                
        except Exception as e:
            print("battle failed:", e)
    return victor


def show_results(names):

    frequency = Counter(names)
    total_count = len(names)

    percentage = {name: (count / total_count) * 100 for name, count in frequency.items()}

    table = []
    table.append(["Name", "Won", "Percentage (%)"])
    for name in frequency:
        table.append([name, frequency[name], round(percentage[name], 2)])

    for row in table:
        print("{:<10} {:<10} {:<10}".format(*row))


In [9]:
BASELINE_RULES=[
    # reglas para la relevancia de deseos
    pr.Rule('desire_h(A,B) <-0 desire_hh(A,B)', 'desire_h_1'),
    pr.Rule('desire_m(A,B) <-0 desire_h(A,B)', 'desire_h_2'),
    pr.Rule('desire_l(A,B) <-0 desire_m(A,B)', 'desire_h_3'),
    pr.Rule('desire_ll(A,B) <-0 desire_l(A,B)', 'desire_h_4'),
    pr.Rule('desire_m(switch,P) <-0 is_switch(switch), is_foe(P, F), on_field_against(C, F), fainted(C)', 'switch_f'),
    
    # reglas para la seleccion de atacar segun efectividad
    pr.Rule('desire_l(move,M) <-0 has_move(P,M), on_field_against(P, F), is_use(move), move_type(M, MT), pokemon_type(F, FT) , effectiveness(MT,FT):[0.2,1.0]', 'move1'),
    pr.Rule('desire_m(move,M) <-0 has_move(P,M), on_field_against(P, F), is_use(move), move_type(M, MT), pokemon_type(F, FT) , effectiveness(MT,FT):[0.4,1.0]', 'move2'),
    pr.Rule('desire_hh(move,M) <-0 has_move(P,M), on_field_against(P, F), is_use(move), move_type(M, MT), pokemon_type(F, FT) , effectiveness(MT,FT):[0.8,1.0]', 'move3'),

    # reglas para el switcheo a un pokemon con mayor ventaja
    pr.Rule('desire_ll(switch,P) <-0 is_foe(P, F), is_switch(switch), has_move(P, M), move_type(M, MT), pokemon_type(F, FT), effectiveness(MT,FT):[0.2,1.0]', 'switch1'),
    pr.Rule('desire_l(switch,P) <-0 is_foe(P, F), is_switch(switch), has_move(P, M), move_type(M, MT), pokemon_type(F, FT), effectiveness(MT,FT):[0.4,1.0]', 'switch2'),
    pr.Rule('desire_m(switch,P) <-0 is_foe(P, F), is_switch(switch), has_move(P, M), move_type(M, MT), pokemon_type(F, FT), effectiveness(MT,FT):[0.8,1.0]', 'switch3'),
]
LESS_SWITCH_RULES=[
    # reglas para la relevancia de deseos
    pr.Rule('desire_h(A,B) <-0 desire_hh(A,B)', 'desire_h_1'),
    pr.Rule('desire_m(A,B) <-0 desire_h(A,B)', 'desire_h_2'),
    pr.Rule('desire_l(A,B) <-0 desire_m(A,B)', 'desire_h_3'),
    pr.Rule('desire_ll(A,B) <-0 desire_l(A,B)', 'desire_h_4'),
    pr.Rule('desire_ll(switch,P) <-0 is_switch(switch), is_foe(P, F), on_field_against(C, F), fainted(C)', 'switch_f'),
    
    # reglas para la seleccion de atacar segun efectividad
    pr.Rule('desire_ll(move,M) <-0 has_move(P,M), on_field_against(P, F), is_use(move), move_type(M, MT), pokemon_type(F, FT) , effectiveness(MT,FT):[0.0,1.0]', 'move1'),
    pr.Rule('desire_l(move,M) <-0 has_move(P,M), on_field_against(P, F), is_use(move), move_type(M, MT), pokemon_type(F, FT) , effectiveness(MT,FT):[0.2,1.0]', 'move1'),
    pr.Rule('desire_m(move,M) <-0 has_move(P,M), on_field_against(P, F), is_use(move), move_type(M, MT), pokemon_type(F, FT) , effectiveness(MT,FT):[0.4,1.0]', 'move2'),
    pr.Rule('desire_hh(move,M) <-0 has_move(P,M), on_field_against(P, F), is_use(move), move_type(M, MT), pokemon_type(F, FT) , effectiveness(MT,FT):[0.8,1.0]', 'move3'),

    # reglas para el switcheo a un pokemon con mayor ventaja
    pr.Rule('desire_ll(switch,P) <-0 is_foe(P, F), is_switch(switch), has_move(P, M), move_type(M, MT), pokemon_type(F, FT), effectiveness(MT,FT):[0.4,1.0]', 'switch2'),
    pr.Rule('desire_l(switch,P) <-0 is_foe(P, F), is_switch(switch), has_move(P, M), move_type(M, MT), pokemon_type(F, FT), effectiveness(MT,FT):[0.8,1.0]', 'switch3'),
]
NO_SWITCH_RULES=[
    # reglas para la relevancia de deseos
    pr.Rule('desire_h(A,B) <-0 desire_hh(A,B)', 'desire_h_1'),
    pr.Rule('desire_m(A,B) <-0 desire_h(A,B)', 'desire_h_2'),
    pr.Rule('desire_l(A,B) <-0 desire_m(A,B)', 'desire_h_3'),
    pr.Rule('desire_ll(A,B) <-0 desire_l(A,B)', 'desire_h_4'),
    pr.Rule('desire_m(switch,P) <-0 is_switch(switch), is_foe(P, F), on_field_against(C, F), fainted(C)', 'switch_f'),
    
    # reglas para la seleccion de atacar segun efectividad
    pr.Rule('desire_ll(move,M) <-0 has_move(P,M), on_field_against(P, F), is_use(move), move_type(M, MT), pokemon_type(F, FT) , effectiveness(MT,FT):[0.0,1.0]', 'move1'),
    pr.Rule('desire_l(move,M) <-0 has_move(P,M), on_field_against(P, F), is_use(move), move_type(M, MT), pokemon_type(F, FT) , effectiveness(MT,FT):[0.2,1.0]', 'move1'),
    pr.Rule('desire_m(move,M) <-0 has_move(P,M), on_field_against(P, F), is_use(move), move_type(M, MT), pokemon_type(F, FT) , effectiveness(MT,FT):[0.4,1.0]', 'move2'),
    pr.Rule('desire_hh(move,M) <-0 has_move(P,M), on_field_against(P, F), is_use(move), move_type(M, MT), pokemon_type(F, FT) , effectiveness(MT,FT):[0.8,1.0]', 'move3'),
    
]

# tournament

In [10]:
cases = {"BASELINE": BASELINE_RULES,
"LESS_SWITCH": LESS_SWITCH_RULES,
"NO_SWITCH": NO_SWITCH_RULES}

In [12]:
for cmb in combinations(cases, 2):
    key1 = cmb[0]
    key2 = cmb[1]
    print("match", key1, "vs", key2)
    player1 = TrainerBDIF(key1, cases[key1])
    player2 = TrainerBDIF(key2, cases[key2])
    result = do_matches(player1, player2, n=11)
    show_results(result)
    print()

match BASELINE vs LESS_SWITCH
mtch: 0
mtch: 1
mtch: 2
mtch: 3
mtch: 4
mtch: 5
mtch: 6
mtch: 7
mtch: 8
mtch: 9
mtch: 10
mtch: 11
mtch: 12
mtch: 13
mtch: 14
mtch: 15
mtch: 16
mtch: 17
mtch: 18
mtch: 19
mtch: 20
mtch: 21
mtch: 22
mtch: 23
mtch: 24
mtch: 25
mtch: 26
mtch: 27
mtch: 28
mtch: 29
mtch: 30
mtch: 31
mtch: 32
mtch: 33
mtch: 34
mtch: 35
mtch: 36
mtch: 37
mtch: 38
mtch: 39
mtch: 40
mtch: 41
mtch: 42
mtch: 43
mtch: 44
mtch: 45
mtch: 46
mtch: 47
mtch: 48
mtch: 49
mtch: 50
mtch: 51
mtch: 52
mtch: 53
mtch: 54
mtch: 55
mtch: 56
mtch: 57
mtch: 58
mtch: 59
mtch: 60
mtch: 61
mtch: 62
mtch: 63
mtch: 64
mtch: 65
mtch: 66
mtch: 67
mtch: 68
mtch: 69
mtch: 70
mtch: 71
mtch: 72
mtch: 73
mtch: 74
mtch: 75
mtch: 76
mtch: 77
mtch: 78
mtch: 79
mtch: 80
mtch: 81
mtch: 82
mtch: 83
mtch: 84
mtch: 85
mtch: 86
mtch: 87
mtch: 88
mtch: 89
mtch: 90
mtch: 91
mtch: 92
mtch: 93
mtch: 94
mtch: 95
mtch: 96
mtch: 97
mtch: 98
mtch: 99
mtch: 100
mtch: 101
mtch: 102
mtch: 103
mtch: 104
mtch: 105
mtch: 106
mtch: 107


# bdi vs llm

In [5]:
inst_old = """-In case any rival pokemon is fainted, still select an action, the rival could send another pokemon, in other case,
if the battle has finished, the combat will end automatically so you wont be prompted to give any further actions.
-The user Will also provide you previous turn decisions and their consequenses can be seen at the next turn state. You are going to account them on your decision so you
avoid repeating mistakes or action taken with panic (like switching back and forth).
-In case there are provided reasoning steps from yourself, you are going to analize them all, chech the most frequent actions, and make a 
choice based on that.
-You will also consider avoid repeating and chaotic switch actions according to previous turns.
When you decide on switch, make sure you won't switch back or make a second switch."""

inst = """
- For each available action, you will make a step by step reasoning, assigning at the end a qualification from 'A'(best possible assignation) to 'F'(worst possible assignation).
- From the previous step set of reasonings, you will make a step by step reasoning to choose an option, based in reasonings and categories assigned.
- From the turn historial, you will try to retrieve information about what pokemons has the rival that are not in the field, so when a pokemon faints, you can choose to switch or prepare
an attack using that information.
- Avoid switching panic (switching between pokemons again and again, use historial to check wether to that current adversary you switched to a specific pokemon, your reasoning took you to
select it).
- You will reassignate categories based on past turn historial (a switch that ended in a bad outcome would be penalized) [examples below]

examples:
use thunderbolt (A) thunderbolt is very effective against POKEMON, so it would be a good option
switch to pikachu (D) pikachu is effective against rival's pokemon, but current pokemon is equally effective, change doesn't make a difference
switch to riolu (B) riolu is equally effective against rival's pokemon, but current pokemon hp is very low, and riolu is on a good state

NOTE: a step by step reasoning in any case, is a list of numered items that represent the steps in the thinking process to select an option.
"""

In [6]:
results_bdi_vs_llm = do_matches(TrainerBDIF("BDI", LESS_SWITCH_RULES), TrainerLLMFeed("LLM", specific_instructions=inst, num_turns=23))

mtch: 0
REASONING FAIL: 429 Resource has been exhausted (e.g. check quota). 
waiting 2 seconds
REASONING FAIL: 429 Resource has been exhausted (e.g. check quota). 
waiting 2 seconds
REASONING FAIL: 429 Resource has been exhausted (e.g. check quota). 
waiting 4 seconds
REASONING FAIL: 429 Resource has been exhausted (e.g. check quota). 
waiting 8 seconds
mtch: 1
REASONING FAIL: 429 Resource has been exhausted (e.g. check quota). 
waiting 2 seconds
REASONING FAIL: 429 Resource has been exhausted (e.g. check quota). 
waiting 4 seconds
REASONING FAIL: 429 Resource has been exhausted (e.g. check quota). 
waiting 2 seconds
REASONING FAIL: 429 Resource has been exhausted (e.g. check quota). 
waiting 2 seconds
mtch: 2
REASONING FAIL: 429 Resource has been exhausted (e.g. check quota). 
waiting 2 seconds
REASONING FAIL: 429 Resource has been exhausted (e.g. check quota). 
waiting 2 seconds
REASONING FAIL: 429 Resource has been exhausted (e.g. check quota). 
waiting 4 seconds
REASONING FAIL: 429

In [7]:
show_results(results_bdi_vs_llm)

Name       Won        Percentage (%)
BDI        17         56.67     
LLM        13         43.33     


# MISC

In [64]:
show_results(results_bdi_vs_llm)

NameError: name 'results_bdi_vs_llm' is not defined

In [ ]:
binomtest(results_bdi_vs_llm.count("BDI"), len(results_bdi_vs_llm), p=0.5, alternative='greater')

In [8]:
bdi = TrainerBDIF("BDI", BASELINE_RULES)
llm = TrainerLLMFeed("LLM", specific_instructions=inst, num_turns=23, talkative= True)
do_battle(bdi, llm, True)

BATTLE STARTED

BDI vs LLM

BDI pokemons are:
roggenrola
oricorio
clefairy
nidoking
blaziken
gardevoir



LLM pokemons are:
tentacruel
anorith
baltoy
ivysaur
genesect
raticate



BDI is thinking...
LLM is thinking...

LLM 's INNER DIALOGUE:
1. **Use knockoff (A)**: Knockoff is super effective against Rock types, and it has a chance to remove the opponent's item. This would be a good option to weaken the opponent and potentially gain an advantage.
2. **Use hyperbeam (C)**: Hyperbeam is super effective against Rock types, but it leaves the user vulnerable for a turn. This is a risky move, but it could potentially take out the opponent in one hit.
3. **Use dive (B)**: Dive is super effective against Rock types, and it avoids all attacks except Surf and Whirlpool. This is a good option if the opponent is likely to use a move that Tentacruel is weak to.
4. **Use reflecttype (F)**: Reflecttype is not effective against Rock types, and it could potentially make Tentacruel vulnerable to attacks

KeyboardInterrupt: Interrupted by user